In [9]:
import pandas as pd

In [10]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [11]:
productos = pd.read_csv("../Productos.csv", names=("product_id", "product_name", "price", "stock"))
pedidos = pd.read_csv("../Pedidos.csv", names=("product_id", "client_id", "qty", "date"))

productos = productos.set_index('product_id')
pedidos['date'] = pd.to_datetime(pedidos['date'], format="%d/%m/%Y")

In [12]:
productos.head()

,product_name,price,stock
product_id,,,
BIT-434,Campana Extractora modelo Glass,375.09,10
SSE-115,Refrigeradora CoolStyle 311N Steel,"3,243.58",23
NMV-644,Lavadora Automatica,"3,272.48",5
MLE-193,Cocina a gas Ingenious,"2,779.41",10
YYK-309,Refrigeradora Door in Door,"4,079.44",25


In [13]:
pedidos.head()

,product_id,client_id,qty,date
0,JXD-139,50375303,6,2023-08-24
1,AWB-345,13245501,1,2023-08-24
2,CRU-009,50375303,5,2023-09-03
3,YYK-309,22777006,3,2023-08-27
4,OTS-581,42157219,5,2023-08-28


In [14]:
# reduce stock from products as orders come

pedidos['qty_served'] = 0
for i, pedido in pedidos.sort_values('date').iterrows():
    pid = pedido['product_id']
    qty = pedido['qty']
    stock = productos.loc[pid, 'stock']
    if stock >= qty:
        pedidos.loc[i, 'qty_served'] = qty
        productos.loc[pid, 'stock'] -= qty
    else:
        pedidos.loc[i, 'qty_served'] = max(0, stock)
        productos.loc[pid, 'stock']  = 0

In [15]:
# reporte

df = pedidos.merge(productos, how='left', on='product_id')

for date, t in df.groupby("date"):
    print("FECHA: ", date.strftime('%d/%m/%Y'))
    t.index = t.reset_index().index + 1
    t['total'] = t['qty_served'] * t['price']
    print(t[['client_id', 'product_id', 'product_name', 'qty_served', 'price', 'total']])

    served_income = sum(t['total'])
    not_served_income = sum((t['qty'] - t['qty_served']) * t['price'])
    print(f"Total ingresado: {served_income:,.2f}")
    print(f"Total perdido por falta de stock: {not_served_income:,.2f}")
    print('========================================================================================================================')

FECHA:  21/08/2023
    client_id product_id                              product_name  qty_served    price     total
1    57690875    CSZ-863                           Video Filmadora           5 1,713.57  8,567.85
2    11602775    CIM-038                   Cocina Empotrable Petra           4 2,544.64 10,178.56
3    29337157    ZEX-027                           Tablet Xperia Z           3 3,843.75 11,531.25
4    49087435    VDD-853  Deshumedecedor  FAD30D7TSSW 3.4L 25-30m2           2 1,095.67  2,191.34
5    62566305    TYF-885                           Videojuego TW 4           4   794.14  3,176.56
6    45877001    VVS-017                           Tablets MW 0731           4 2,599.72 10,398.88
7    83505958    EJW-346                        Campana Extractora           3   359.70  1,079.10
8    98140264    QHC-492                            Terma Solucion           4 1,907.23  7,628.92
9    40348917    WZD-331                Refrigeradora ECOTT 465 IX           3 1,213.79  3,641.37
1

In [16]:
productos

,product_name,price,stock
product_id,,,
BIT-434,Campana Extractora modelo Glass,375.09,0
SSE-115,Refrigeradora CoolStyle 311N Steel,"3,243.58",10
NMV-644,Lavadora Automatica,"3,272.48",0
MLE-193,Cocina a gas Ingenious,"2,779.41",6
YYK-309,Refrigeradora Door in Door,"4,079.44",13
...,...,...,...
QFP-254,Calentador electrico instantaneo,180.65,0
DUG-251,Cocina a gas FFG3047SL,"1,597.96",0
III-576,Campana Extractora Venus Plus,"1,077.65",5
